## Statsmodel Forecast with Wallaroo Features: Deploy and Test Infer

This tutorial series demonstrates how to use Wallaroo to deploy a statsmodel ARIMA forecast model and perform sample inferences through it.

In the previous step "Statsmodel Forecast with Wallaroo Features: Model Creation", the statsmodel was trained and saved to the Python file `forecast.py`.  This file will now be uploaded to a Wallaroo instance as a Python model, then used for sample inferences.

## Prerequisites

* A Wallaroo instance version 2023.2.1 or greater.

## References

* [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Python Models](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-upload-python/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)
* [Wallaroo SDK Essentials: Inference Guide: Parallel Inferences](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-inferences/#parallel-inferences)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries that we will need.

In [4]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import pyarrow as pa

In [5]:
wallaroo.__version__

'2023.2.1'

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [6]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

# wallarooPrefix = "doc-test."
# wallarooSuffix = "wallaroocommunity.ninja"

wallarooPrefix = "product-uat-ee."
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Set Configurations

The following will set the workspace, model name, and pipeline that will be used for this example.  If the workspace or pipeline already exist, then they will assigned for use in this example.  If they do not exist, they will be created based on the names listed below.

Workspace names must be unique.  To allow this tutorial to run in the same Wallaroo instance for multiple users, set the `suffix` variable or share the workspace with other users.

#### Set Configurations References

* [Wallaroo SDK Essentials Guide: Workspace Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-workspace/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)

In [7]:
# used for unique connection names

# import string
# import random

# suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))

suffix='jch4'

workspace_name = f'forecast-model-workshop{suffix}'

pipeline_name = 'forecast-workshop-pipeline'

### Set the Workspace and Pipeline

The workspace will be either used or created if it does not exist, along with the pipeline.

In [8]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

### Upload Model

The Python model created in "Forecast and Parallel Infer with Statsmodel: Model Creation" will now be uploaded.  Note that the Framework is set to `Framework.PYTHON` to inform the Wallaroo engine on what model framework is being added.

The other versions of the model are also uploaded in this step to show how to hot swap between models and other uses.

In [9]:
# upload three models:  the control and two challengers

control_model_name = 'forecast-control-model'
control_model_file = './forecast_standard.py'

challenger01_model_name = 'forecast-challenger01-model'
challenger01_model_file = './forecast_alternate01.py'

challenger02_model_name = 'forecast-challenger02-model'
challenger02_model_file = './forecast_alternate02.py'

# Holding on these for later
# input_schema = pa.schema([
#     pa.field('count', pa.list_(pa.int64(), list_size=7))
# ])

input_schema = pa.schema([
    pa.field('count', pa.int64())
])
# output_schema = pa.schema([
#     pa.field('forecast', pa.list_(pa.int64()))
# ])

output_schema = pa.schema([
    pa.field('forecast', pa.int64())
])

# upload the models

bike_day_model = wl.upload_model(control_model_name, 
                                 control_model_file, 
                                 input_schema=input_schema,
                                 output_schema=output_schema,
                                 framework = Framework.PYTHON)

# bike_day_model = wl.upload_model(control_model_name, control_model_file, Framework.PYTHON)

# challenger_model_01 = wl.upload_model(challenger01_model_name, challenger01_model_file, Framework.PYTHON)

# challenger_model_02 = wl.upload_model(challenger02_model_name, challenger02_model_file, Framework.PYTHON)



### Deploy the Pipeline

We will now add the uploaded model as a step for the pipeline, then deploy it.

Until a pipeline is deployed, the steps assigned to it only exist in the local memory.  During deployment the pipeline steps, configurations, and other details are set in the database, and resources allocated from the cluster for the pipeline's use.

In [10]:
# Set the deployment to allow for additional engines to run
# Undeploy and clear the pipeline in case it was used in other demonstrations
pipeline.undeploy()
pipeline.clear()
deploy_config = (wallaroo.DeploymentConfigBuilder()
                        .replica_count(1)
                        .replica_autoscale_min_max(minimum=2, maximum=5)
                        .cpus(0.25)
                        .memory("512Mi")
                        .build()
                    )

pipeline.add_model_step(bike_day_model)
# pipeline.add_model_step(step)
# pipeline.deploy(deployment_config = deploy_config)
pipeline.deploy()

WaitForDeployError: Deployment failed. See status for details.
Status: {'status': 'Error', 'details': [], 'engines': [{'ip': None, 'name': 'engine-576b794cff-dpcjl', 'status': 'Pending', 'reason': '0/9 nodes are available: 9 Insufficient cpu. preemption: 0/9 nodes are available: 9 No preemption victims found for incoming pod.', 'details': ['0/9 nodes are available: 9 Insufficient cpu. preemption: 0/9 nodes are available: 9 No preemption victims found for incoming pod.'], 'pipeline_statuses': None, 'model_statuses': None}], 'engine_lbs': [{'ip': '10.244.8.3', 'name': 'engine-lb-584f54c899-2vmsj', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}

In [ ]:
pipeline.steps()

[{'ModelInference': {'models': [{'name': 'forecast-control-model', 'version': '14df2c41-91e4-4077-ad82-2ed6e298132b', 'sha': '4e4d344bc5496817092f550e664eee0445051e2dcce4527c6c2dbe6685275a90'}]}}]

### Run Inference

Run a test inference to verify the pipeline is operational from the sample test data stored in `./data/testdata_dict.json`.

In [ ]:
inferencedata = pd.read_json('./data/testdata_standard.df.json')
# display(inferencedata)

# display(pa.Schema.from_pandas(inferencedata))

inference_frame = pd.DataFrame({"count": [15]})

results = pipeline.infer(inference_frame)

# display(results)

KeyError: 'time'

### Undeploy the Pipeline

Undeploy the pipeline and return the resources back to the Wallaroo instance.

In [ ]:
pipeline.undeploy()

name,forecast-workshop-pipeline
created,2023-07-27 21:04:23.296526+00:00
last_updated,2023-07-27 21:04:23.296526+00:00
deployed,False
tags,
versions,99154217-cc5d-4458-8061-be427767e988
steps,forecast-control-model
